<font face="Arial" size=6 color=#0099ff> 0. Importing packages </font>

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. Organize packages in order (system and io packages, basic packages, statistics packages, plot packages, other packages.)
2. ALWAYS add and revise on the same template and put it into Dash snippets. 

<font face="Arial" size=5 color=#DC143C> why: </font>
1. easy to manage packages in the enviroment (clear overview of what are in the running enviroment)
2. make your code tidy 

In [ ]:
# system and io packages
import sys
import cPickle as pickle

#Basic packages
import pandas as pd
import numpy as np

#statistics packages
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import spearmanr
import statsmodels.api as sm
from statsmodels.formula.api import ols
%load_ext rpy2.ipython
from rpy2.robjects import r, pandas2ri

#plot packages
import matplotlib
import matplotlib.pyplot as plt
from pylab import *
import seaborn as sns
%matplotlib inline

import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot,iplot_mpl
from plotly.graph_objs import *
import cufflinks as cf
init_notebook_mode(connected=True)

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets



# other packages
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

<font face="Arial" size=6 color=#0099ff> 1.Import and combine Edata and basic demography data（age,gender...)
</font> 
<font face="Arial" size=5 color=#DC143C> tips: </font>
1. load and save data by using read_csv() and to_csv() 
2. merge table by using df1.set_index(’Subject’).join(df2.set_index(’Subject’)) by index, also can use 'on=column name' 
3. setting 'Subject ID' as index is convenient 
4. after manipulating data (e.g. join, merge, groupby), ALWAYS check the resulting tables by using df.describe() and head()

 **Import behavioral data(excel version of Edata)**

In [ ]:
# read excel version of Edata, pandas.read_excel
se_all = pd.read_excel('SE_all.xlsx')
raw_data = se_all

In [ ]:
#atb_list = ['ExperimentName','Subject','Block','answer.ACC','answer.CRESP',  
#            'Trigger.OnsetTime','answer.OnsetTime','answer.RESP','answer.RT',  
#            'answer.RTTime','answer1','answer2','answer3','answer4','condition',
#            'FA','repot.RESP','repot.RT','repot.RTTime','rightA',  
#            'run','sen','sen_no','word']

In [ ]:
#set_index, rename, join
data = raw_data.set_index(['Subject'])
# replace wrong named data including {63:20,99:11,152:15,217:17}
replaced_pairs = {63:20,99:11,152:15,217:17}
data = data.rename(replaced_pairs, axis='index')
se_basic_infor = pd.read_excel('SE_basic_infor.xlsx')
se_basic_infor = se_basic_infor.set_index('Subject')
data = data.join(se_basic_infor)
data.describe()

Suject 24(absent),53(not follow the instruction),99,152,217 not clear what is the issue

<font face="Arial" size=6 color=#0099ff>  2. Check response data (RT and response) for all the trials from each subject </font>

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. before average RTs cross trials, look into the distribution (pattern) of trial-trial RTs. detect outliers and address them properly (subject 52)
2. use @interact from ipywidgets to check the histogram interactively. 

In [ ]:
#I think that we should not remove acc very low, maybe false alarm very high.
#52, 53, 54, 56 for pre, 
# for unp 3, 11, 14, 15, 17, 18, 19, 20, 21, 23, 25, 26, 31, 32, 33, 36, 39,
#            40, 41, 42, 44, 45, 47, 50, 52, 54, 56, 57, 58, 60, 61, 62
# opt['acc_pre'] = bd_pre['answer.ACC'].mean(level='Subject')
# opt[opt['acc_unp']<0.25].index 
# opt['fa_unp']/60
#sub42(30 zeros)，44(60 zeros)，52(90 and more zeros)
@interact(index_name=data.index.unique())
def iplot_scale(index_name):
    df = data[data.index==index_name]['answer.RT']
    iplot(df.iplot(asFigure=True,kind='histogram', bins = 20,subplots=True, shape=(1, 1), filename='histogram-subplots'))


<font face="Arial" size=6 color=#0099ff>  3. Average RT and ACC for each subject. </font>

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. df[df['column_name']] to select column
2. df.mean(level='Sujbect') to average rt for each subject cross trials. df.fun() (sum, std, sqrt...)to manipulate the data. 
3. pd.concat((df1,df2),axis=1) to concat RT, ACC, False alarm and other variables into one table. 

In [ ]:
#to check RT distribution for each subject and detect outliers
opt =  pd.DataFrame()
bd_pre =  data[data['condition']=='pre']
bd_unp =  data[data['condition']=='unp']
# rt is zero in some trials, exluding them.

opt['acc_pre'] = bd_pre['answer.ACC'].mean(level='Subject')
opt['acc_unp'] = bd_unp['answer.ACC'].mean(level='Subject')


# only for all RIGHT response 
opt['conf_pre'] = bd_pre[bd_pre['answer.ACC']==1]['answer.RESP'].mean(level='Subject')
opt['conf_unp'] = bd_unp[bd_unp['answer.ACC']==1]['answer.RESP'].mean(level='Subject')

# only for all RIGHT response 
#opt['conf_pre'] = bd_pre['answer.RESP'].mean(level='Subject')
#opt['conf_unp'] = bd_unp['answer.RESP'].mean(level='Subject')


opt['rt_pre'] = bd_pre[bd_pre['answer.ACC']==1]['answer.RT'].mean(level='Subject') #
opt['rt_unp'] = bd_unp[bd_unp['answer.ACC']==1]['answer.RT'].mean(level='Subject') #[bd_unp['answer.ACC']==1]
#opt['rt_sd_pre'] = bd_pre[bd_pre['answer.ACC']==1]['answer.RT'].std(level='Subject')
#opt['rt_sd_unp'] = bd_unp[bd_unp['answer.ACC']==1]['answer.RT'].std(level='Subject')

# add false alarm rate
bd_unp_ans_fa = bd_unp[bd_unp['answer.RESP']==bd_unp['FA']][['answer.RESP']]
bd_unp_ans_fa['count_num'] = 1
opt['fa_unp'] = bd_unp_ans_fa['count_num'].sum(level='Subject')
opt['fa_pre'] = 0

#concat different condtions and different measurements. 

acc = pd.concat((opt['acc_pre'],opt['acc_unp']),axis = 0)  
conf = pd.concat((opt['conf_pre'],opt['conf_unp']),axis = 0)                  
rt = pd.concat((opt['rt_pre'],opt['rt_unp']),axis = 0) 
fa = pd.concat((opt['fa_pre'],opt['fa_unp']),axis = 0) 
opt_all = pd.concat((acc,rt,fa,conf),axis=1)
opt_all = opt_all.rename({0:'acc',1:'rt',2:'fa',3:'conf'},axis = 'columns')
opt_all['condition'] = ['pre']*61+['unp']*61
opt_all = opt_all.join(se_basic_infor)
opt_all.describe()

**Based on scale score to devide all subject into two groups. add a group variable. **

In [ ]:
opt = opt.join(se_basic_infor)
#print(opt)
LSHS_AVH_median = opt['LSHS_AVH'].median()
group = []
for i in opt.index:
    if opt['LSHS_AVH'][i]<=LSHS_AVH_median:
        group.append('Low')
    else:
        group.append('High')
opt['group'] = group 
opt.dropna(how='any')
#opt[['LSHS_AVH','group']]
opt_all = opt_all.join(opt['group'])

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. df.iloc[i,j] to select the subset of the data.

In [ ]:
# combine with questionaire data

se_scale = pd.read_excel('SE_scale.xlsx')
se_scale  = se_scale.set_index('Subject')
#print(se_scale)
#se_scale = se_scale.set_index(['Subject'])
se_scale = se_scale.iloc[:,7:76]
opt_all = opt_all.join(se_scale)
se_qpe = pd.read_excel('SE-QPE_40.xlsx')
se_qpe = se_qpe.set_index('Subject')
QPE_severity = pd.DataFrame(se_qpe.iloc[:,12:16].sum(axis = 1),columns = ['QPE_severity'])
opt_all = opt_all.join(QPE_severity)

<font face="Arial" size=6 color=#0099ff>  4. get sum scores of all iterms for each questionires. </font>

In [ ]:
# calculate and summary the sales into sum scores.
#print(opt_all.columns)
stai_all = range(1,21)
stai_all_opp = [1,3,6,7,10,13,14,16,19]
stai_all_pos = list(set(stai_all).difference(set(stai_all_opp)))

STAI_oppsiteiterm_colnames = ['STAI'+str(i) for i in stai_all_opp]
STAI_positerm_colnames = ['STAI'+str(i) for i in stai_all_pos]
opt_all['TAI_all'] = 45-opt_all.loc[:,STAI_oppsiteiterm_colnames].sum(axis=1)+opt_all.loc[:,STAI_positerm_colnames].sum(axis=1)
#print(opt_all.loc[:,STAI_positerm_colnames])
#print(x)
BDI_colnames = ['BDI'+str(i+1) for i in range(21)]
HPSV_colnames = ['HPSV'+str(i+1) for i in range(9)]
RAHQ_colnames = ['RAHQ'+str(i+1) for i in range(18)]

opt_all['BDI_all'] = opt_all.loc[:,BDI_colnames].sum(axis=1)
opt_all['HPSV_all'] = opt_all.loc[:,HPSV_colnames].sum(axis=1)
opt_all['RAHQ_all'] = opt_all.loc[:,RAHQ_colnames].sum(axis=1)

**Drop some subjects.**

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. df.drop() to delete column or run 

In [ ]:
opt_all = opt_all.drop([53,34])# also remove sub34 because no avh basic infors.
#rt, zero exist. sub42(30 zeros)，44(60 zeros)，52(90 and more zeros)
#print(opt_all[opt_all['condition']=='pre']['acc'])
#opt_all[np.abs(opt_all.acc-opt_all.acc.mean())<=(3*opt_all.acc.std())] 
#opt_all[~(np.abs(opt_all.acc-opt_all.acc.mean())>(3*opt_all.acc.std()))] 
#opt_all[opt_all.acc<0.25]

<font face="Arial" size=6 color=#0099ff>  5. check distribution and individual data points for gender, age and questionnaires </font>

In [ ]:
df = opt_all[['age','group']]
iplot(df['age'].iplot(asFigure=True,kind='histogram', bins = 20,subplots=True, shape=(1, 1), filename='histogram-subplots'))

**Check distribution and correlation among scale scores**

In [ ]:
# df_bygroup_bd = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all','group']].groupby(['group']).mean()
# y_err_bd = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all','group']].groupby(['group']).sem()
# df_bygroup_bd = df_bygroup_bd.T
# y_err_bd = y_err_bd.T

# df_bygroup_bd.plot(kind = 'bar',yerr = y_err_bd,width=0.8,style=['r','b'])#yerr = y_err_v,
# plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
# #ylabel('bd')
# show()

In [ ]:
df = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all']]
iplot(df.iplot(asFigure=True,kind='histogram', bins = 20,subplots=True, shape=(7, 1), filename='histogram-subplots'))

In [ ]:
df_all = df.drop_duplicates()
print(df_all.columns)
@interact(scale_name=df_all.columns)
def iplot_scale(scale_name):
    df = df_all[scale_name]
    iplot(df.iplot(asFigure=True, kind='histogram', barmode='stack', bins=20, filename='basic-histogram'))

In [ ]:
@interact(scale_name=df_all.columns)
def iplot_scale(scale_name):
    d = np.array(df_all[scale_name].dropna())
    print(d)
    sns.set(style="white", palette="muted", color_codes=True)

    # Set up the matplotlib figure
    f, axes = plt.subplots(figsize=(7, 7), sharex=True)
    sns.despine(left=True)
    # Plot a historgram and kernel density estimate
    sns.distplot(d, color="m")
    plt.setp(axes, yticks=[])
    plt.tight_layout()

In [ ]:
df = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity']]#,'TAI_all','BDI_all'
iplot(df.scatter_matrix(asFigure=True,filename='scatter-matrix-subplot', bins=20, world_readable=True))

**Check distribution and correlation among scale scores**

In [ ]:
df = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','TAI_all','BDI_all']]#
iplot(df.scatter_matrix(asFigure=True,filename='scatter-matrix-subplot', bins=20, world_readable=True))
#help(df.scatter_matrix)

**Plot the distribution of ACC by condition and group**

In [ ]:
df_groupby = opt_all[['acc','group','condition']].groupby(['group','condition'])
print(df_groupby.groups.keys())
df_list = [df_groupby.get_group(key).reset_index()['acc'].rename() for key in df_groupby.groups.keys()]
df = pd.concat(df_list,axis=1)
#df = df.rename(columns={1:})
iplot(df.iplot(asFigure=True,kind='histogram', barmode = 'stack',bins = 20, filename='histogram-subplots'))

<font face="Arial" size=6 color=#0099ff>  6. Calculate and plot variables in group level (gender,age,questionaires,rt,acc and other) </font>

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. use ** df.groupby() ** to devide data into sub-groups, then use formula (function) (mean, sem...) to manipulate data for each sub-group.

In [ ]:
df_bygroup_basicinfor = opt_all[['gender','age','group']].groupby(['group']).mean()
y_err_bd = opt_all[['gender','age','group']].groupby(['group']).sem()
df_bygroup_basicinfor = df_bygroup_basicinfor.T
y_err_bd = y_err_bd.T

df_bygroup_basicinfor.plot(kind = 'bar',yerr = y_err_bd,width=0.8,style=['r','b'])#yerr = y_err_v,
plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
#ylabel('bd')
show()

#print(opt_all[['gender','group']])
df_bygroup_basicinfor = opt_all[['age','gender','group']].groupby(['gender','group']).count()/2
print(df_bygroup_basicinfor)
#y_err_bd = opt_all[['gender','group']].groupby(['group']).sem()
df_bygroup_basicinfor = df_bygroup_basicinfor.T
#y_err_bd = y_err_bd.T

df_bygroup_basicinfor.plot(kind = 'bar',width=0.8,style=['r','b'])#yerr = y_err_v,
plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
#ylabel('bd')
show()

**Plot scale scores between groups**

In [ ]:
df = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all','group']]
df = df.drop_duplicates()
# color_list = ['rgba(93, 164, 214, 0.5)', 'rgba(93, 164, 214, 1)',  
#               'rgba(44, 160, 101, 0.5)', 'rgba(44, 160, 101, 1)',  
#               'rgba(255, 65, 54, 0.5)', 'rgba(255, 65, 54, 1)',  
#               'rgba(207, 114, 255, 0.5)','rgba(207, 114, 255, 1)',  
#               'rgba(127, 96, 0, 0.5)','rgba(127, 96, 0, 1)',  
#               'rgba(255, 165, 0,0.5)','rgba(255, 165, 0,1)',  
#               'rgba(255, 20, 147,0.5)','rgba(255, 20, 147,1)']
color_list = ['rgba(93, 164, 214, 5)', 'rgba(93, 214, 214, 1)',  
              'rgba(44, 160, 101, 5)', 'rgba(44, 210, 101, 1)',  
              'rgba(255, 65, 54, 5)', 'rgba(255, 115, 54, 1)',  
              'rgba(207, 114, 255, 5)','rgba(207, 164, 255, 1)',  
              'rgba(127, 96, 0, 5)','rgba(127, 146, 0, 1)',  
              'rgba(255, 165, 0,5)','rgba(255, 215, 0,1)',  
              'rgba(255, 20, 147,5)','rgba(255, 70, 147,1)']
scale_list = ['LSHS_total','LSHS_total','LSHS_AVH','LSHS_AVH','HPSV_all','HPSV_all','RAHQ_all','RAHQ_all',  
              'QPE_severity','QPE_severity','TAI_all','TAI_all','BDI_all','BDI_all']
group_list = ['High','Low']*7
df_gls = pd.DataFrame({'gls':group_list,
                       'sls':scale_list,
                       'cls':color_list})
data = [
            {
                'y': df[df['group']==group][clms],
                'type': 'box', 
                'boxpoints':'all',
                #'fillcolor':
                'marker': dict(color = df_gls[(df_gls['gls']==group)&(df_gls['sls']==clms)]['cls'].values[0]),
                'boxmean':'sd',
                'name': clms + '_' + group
            } 
                for clms in ['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all']
            for group in ['High','Low']
        ]
iplot(Figure(data=Data(data),layout = Layout(showlegend=False)))
#py.image.save_as(Figure(data=Data(data)), filename='a-simple-plot.jpeg')

**Calculate difference of scale scores between groups**

In [ ]:
# get a list of all columns in the dataframe without the Group column
df = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all','group']]
df = df.drop_duplicates()
column_list = [x for x in df.columns if x != 'group']
# create an empty dictionary
t_test_results = {}
# loop over column_list and execute code explained above
for column in column_list:
    group1 = df.where(df.group== 'High')[column].dropna()
    group2 = df.where(df.group== 'Low')[column].dropna()
    print group1.count(),group2.count()
    # add the output to the dictionary 
    t_test_results[column] = ttest_ind(group1,group2)
results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
results_df.columns = ['statistic','pvalue']
results_df

<font face="Arial" size=6 color=#0099ff>  7.Plot bar graph and do anova and t-test statistics for RT and ACC. </font>

**Plot acc by conditon and group**

In [ ]:
# remove subjects with very high fa, let's say more than 40
opt_all = opt_all[~opt_all.index.isin(opt_all[opt_all['fa']>40].index)]
df_bygroup_acc = opt_all[['acc','group','condition']].groupby(['group','condition']).mean()
y_err_acc = opt_all[['acc','group','condition']].groupby(['group','condition']).sem()
df_bygroup_acc = df_bygroup_acc.T
y_err_acc = y_err_acc.T

df_bygroup_acc.plot(kind = 'bar',yerr = y_err_acc,width=0.8,style=['r--','r','b--','b'])#yerr = y_err_v,
plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
#ylabel('acc')
show()

In [ ]:
#print opt_all
opt_all_tmp = opt_all.copy()
opt_all_tmp.reset_index(inplace=True)
sublist_all = opt_all_tmp[['Subject','group']].drop_duplicates()
sublist_paired = pd.read_table('Test_Preproc_GHY_SE.txt',header=None)
#?pd.read_table
sublist_paired.columns=['DNA_No','DNA_format_No']
sublist_paired['DNA_No'] = sublist_paired['DNA_No'].apply(lambda x: int(x[4:]))
sublist_paired = sublist_paired.rename(columns={'DNA_No':'Subject'})
sublist_FA40_all = sublist_all.set_index('Subject').join(sublist_paired.set_index('Subject'))
sublist_FA40_all[sublist_FA40_all.group=='High']['DNA_format_No'].to_csv('sublist_high_all.txt',index=False)
sublist_FA40_all[sublist_FA40_all.group=='Low']['DNA_format_No'].to_csv('sublist_low_all.txt',index=False)

**ANOVA for ACC**


<font face="Arial" size=5 color=#DC143C> tips: </font>
1. organize the data in the format requred by R fomula
2. use rpy2 interface to use R
3. use pivot to transform long format into wide format data, then save csv data to check statistic by using JASP.

In [ ]:
df_se_pra = opt_all.copy()
df_se_pra.reset_index(inplace=True)
for bd_val in ['acc']:
    print('This is anova for %s:') %(bd_val)
    df_bd_val = df_se_pra[['Subject','group','condition',bd_val]]#

    df_bd_val = df_bd_val.drop_duplicates()
    # remove all subjects including na
    sublist_na = df_bd_val['Subject'][df_bd_val.iloc[:,-1].isna()].values
    df_bd_val = df_bd_val[~df_bd_val['Subject'].isin(sublist_na)]
    #print df_bd_val
#     sublist_bls = df[df.duplicated(keep='first')]['Subject'].values
#     df_bd_val = df_bd_val[df_bd_val['Subject'].isin(sublist_bls)]    
    #df2=df_bd_val.pivot(index='Subject', columns='condition', values='rt')
    
    r_df_bd_val = pandas2ri.py2ri(df_bd_val) 
    #print(r_df_bd_val)
    if bd_val == 'acc':
            %R -i r_df_bd_val r_df_bd_val$Subject <-factor(r_df_bd_val$Subject);\
            aov_group_conditon <- aov(acc ~ group*condition + Error(Subject/condition), data=r_df_bd_val);\
            print(summary(aov_group_conditon));print(model.tables(aov_group_conditon, "means"))
   
    # tranform data from long format into wide format. 
    df2=df_bd_val.pivot(index='Subject', columns='condition', values='acc')
    df2 = df2.join(df_bd_val[df_bd_val.condition=='pre'][['Subject','group']].set_index('Subject'))
    #print df2
    df2.to_csv('df_acc4anova.csv')

**t-test for ACC to check interaction pattern (simple effect)**

In [ ]:
for con in ['pre','unp']:
    df = opt_all[['acc','group','condition']]
    df = df[df['condition']==con][['acc','group']]
    #print(df)
    column_list = [x for x in df.columns if x != 'group']
    # create an empty dictionary
    t_test_results = {}
    # loop over column_list and execute code explained above
    for column in column_list:
        group1 = df.where(df.group== 'High').dropna()[column]
        group2 = df.where(df.group== 'Low').dropna()[column]
        print group1.count(),group2.count()
        # add the output to the dictionary 
        #print(group1,group2)
        t_test_results[column] = ttest_ind(group1,group2)
    results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
    results_df.columns = ['statistic','pvalue']
    print con,'\n',results_df

In [ ]:
df_bygroup_rt = opt_all[['rt','group','condition']].groupby(['group','condition']).mean()
y_err_rt = opt_all[['rt','group','condition']].groupby(['group','condition']).sem()
df_bygroup_rt = df_bygroup_rt.T
y_err_rt = y_err_rt.T

df_bygroup_rt.plot(kind = 'bar',yerr = y_err_rt,width=0.8,style=['r--','r','b--','b'])#yerr = y_err_v,
plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
#ylabel('acc')
show()

In [ ]:
df_se_pra = opt_all.copy()
df_se_pra.reset_index(inplace=True)
for bd_val in ['rt']:
    print('This is anova for %s:') %(bd_val)
    df_bd_val = df_se_pra[['Subject','group','condition',bd_val]]#
    df_bd_val = df_bd_val.drop_duplicates()
    # remove all subjects including na
    sublist_na = df_bd_val['Subject'][df_bd_val.iloc[:,-1].isna()].values
    df_bd_val = df_bd_val[~df_bd_val['Subject'].isin(sublist_na)]
#     sublist_bls = df[df.duplicated(keep='first')]['Subject'].values
#     df_bd_val = df_bd_val[df_bd_val['Subject'].isin(sublist_bls)]    
    #df2=df_bd_val.pivot(index='Subject', columns='condition', values='rt')
    df_bd_val.to_csv('tmp.csv')
    r_df_bd_val = pandas2ri.py2ri(df_bd_val) 
     #print(r_df_bd_val)
    if bd_val == 'rt':
        %R -i r_df_bd_val r_df_bd_val$Subject <-factor(r_df_bd_val$Subject);aov_group_conditon <- aov(rt ~ group*condition + Error(Subject/condition), data=r_df_bd_val);print(summary(aov_group_conditon));print(model.tables(aov_group_conditon, "means"))
    

In [ ]:
for con in ['pre','unp']:
    df = opt_all[['rt','group','condition']]
    df = df[df['condition']==con][['rt','group']]
    #print(df)
    column_list = [x for x in df.columns if x != 'group']
    # create an empty dictionary
    t_test_results = {}
    # loop over column_list and execute code explained above
    for column in column_list:
        group1 = df.where(df.group== 'High').dropna()[column]
        group2 = df.where(df.group== 'Low').dropna()[column]
        print group1.count(),group2.count()
        # add the output to the dictionary 
        #print(group1,group2)
        t_test_results[column] = ttest_ind(group1,group2)
    results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
    results_df.columns = ['statistic','pvalue']
    print con, '\n',results_df

In [ ]:
for bd_val  in ['acc','rt']:
    df = opt_all[[bd_val,'group','condition']]
    df = df.drop_duplicates()
    color_list = ['rgba(255, 65, 54, 5)', 'rgba(255, 115, 54, 1)',  
                  'rgba(44, 160, 101, 5)', 'rgba(44, 210, 101, 1)']
    scale_list = ['unp','pre','unp','pre']
    group_list = ['High']*2+['Low']*2
    df_gls = pd.DataFrame({'gls':group_list,
                           'sls':scale_list,
                           'cls':color_list})
    #print(df_gls)
#     for group in ['High','Low']:
#         for clms in ['unp','pre']:
#             print(group,clms)
            #print(df_gls[(df_gls['gls']==group)&(df_gls['sls']==clms)]['cls'].values[0])

    data = [
                {
                    'y': df[(df['group']==group)&(df['condition']==clms)][bd_val],
                    'type': 'box', 
                    'boxpoints':'all',
                    #'fillcolor':
                    'marker': dict(color = df_gls[(df_gls['gls']==group)&(df_gls['sls']==clms)]['cls'].values[0]),
                    'boxmean':'sd',
                    'name': clms + '_' + group
                }   
                    for group in ['High','Low']
                for clms in ['unp','pre']
            ]
    iplot(Figure(data=Data(data),layout = Layout(width=500,height=500,title=bd_val,showlegend=True)))


<font face="Arial" size=5 color=#DC143C> use dPrime function to calulate singal detection theroy measures, d,beta,c,Ad. </font> 

In [ ]:
from __future__ import division # Not neccessary in Python 3 and later
from scipy.stats import norm
from math import exp,sqrt
Z = norm.ppf
 
def dPrime(hits, misses, fas, crs):
    # Floors an ceilings are replaced by half hits and half FA's
    halfHit = 0.5/(hits+misses)
    halfFa = 0.5/(fas+crs)
 
    # Calculate hitrate and avoid d' infinity
    hitRate = hits/(hits+misses)
    if hitRate == 1: hitRate = 1-halfHit
    if hitRate == 0: hitRate = halfHit
 
    # Calculate false alarm rate and avoid d' infinity
    faRate = fas/(fas+crs)
    if faRate == 1: faRate = 1-halfFa
    if faRate == 0: faRate = halfFa
 
    # Return d', beta, c and Ad'
    out = {}
    out['d'] = Z(hitRate) - Z(faRate)
    out['beta'] = exp((Z(faRate)**2 - Z(hitRate)**2)/2)
    out['c'] = -(Z(hitRate) + Z(faRate))/2
    out['Ad'] = norm.cdf(out['d']/sqrt(2))
    return out
dPrime(0.8,0.2,0.7,0.3)

In [ ]:
#opt_all = opt_all.set_index('Subject')
opt_all['fa'] = opt_all['fa'].apply(lambda x:x/60)
acc_2cond = opt_all[['group','condition','fa','acc']].groupby(['Subject'])['acc'].mean()
opt_all.reset_index(inplace=True) 
opt_all =opt_all.set_index(['Subject'])
opt_all = opt_all.join(acc_2cond,rsuffix='_2cond')
opt_all.reset_index(inplace=True) 
opt_all =opt_all.set_index(['Subject'])
opt_all.reset_index(inplace=True) 
df_dPrime = [dPrime(opt_all['acc_2cond'][idx],1-opt_all['acc_2cond'][idx],opt_all['fa'][idx],1-opt_all['acc_2cond'][idx]) for idx in opt_all.index]
opt_all = opt_all.join(pd.DataFrame(df_dPrime))
opt_all = opt_all.set_index('Subject')

In [ ]:
#opt_all[['group','condition','d','c','Ad','beta','fa']].to_csv('tmp.csv')
opt_all.reset_index(inplace=True)
#opt_all = opt_all[opt_all['fa']<=40/60]

In [ ]:
# themes =  cf.getThemes()
# print(themes)
# cf.set_config_file(theme=themes[6])
#print(opt_all)
bd_str = 'fa'
#opt_all.to_csv('tmp1.csv')
sublist_na = opt_all['Subject'][opt_all.loc[:,bd_str].isna()].values
df_se_pra_nona = opt_all[~opt_all['Subject'].isin(sublist_na)]
df_tmp = df_se_pra_nona[(df_se_pra_nona['condition']=='unp')][['Subject','group','d','c','Ad','beta','fa']]
df_tmp = df_tmp.drop_duplicates()
df_tmp.to_csv('tmp.csv')
data = [
            {
                'y': df_tmp[(df_tmp['group']==group)][bd_str],
                'type': 'box', 
                'boxpoints':'all',
                'name': bd_str + '_' + str(group)
            } 
                for bd_str in  ['d','c','Ad','beta']
            for group in ['High','Low']
        ]
iplot(Figure(data=Data(data)))

**correlation between scales score and behaviorial measurements**

In [ ]:
#%debug
#bd_str = 'LSHS_AVH'

#opt_all.to_csv('tmp1.csv')
sublist_na = opt_all['Subject'][(opt_all.loc[:,'BDI_all'].isna())|(opt_all.loc[:,'LSHS_AVH'].isna())].values
df_se_pra_nona = opt_all[~opt_all['Subject'].isin(sublist_na)]
df_tmp = df_se_pra_nona[(df_se_pra_nona['condition']=='unp')][['Subject','LSHS_total','LSHS_AVH',  
                                                               'HPSV_all','RAHQ_all','QPE_severity',  
                                                               'TAI_all','BDI_all','d',  
                                                               'Ad','c','beta','fa']]  
                                                               
df = df_tmp.drop_duplicates()
df.to_csv('tmp.csv')
from scipy.stats import pearsonr
#print(pearsonr(df['LSHS_total'],df['d']))
r_p = [pearsonr(df[cls1],df[cls2]) for cls1 in df.columns for cls2 in df.columns]
r, p = zip(*r_p)
bd_names = ['Subject','LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all',  
            'QPE_severity','TAI_all','BDI_all','d','Ad','c','beta','fa']
r = pd.DataFrame(np.array(r).reshape((13,13)),columns=bd_names,index = bd_names)
p = pd.DataFrame(np.array(p).reshape((13,13)),columns=bd_names,index = bd_names)
print(p[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity']])
#print(p)


import seaborn as sns; sns.set(color_codes=True)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 4, figsize=(15, 15))
sns.despine(left=True)
cnt1 = 0
for clms1 in ['LSHS_AVH','RAHQ_all']:
    cnt2 = 0
    for clms2 in ['d','Ad','c','beta']:#,'fa'
        b= sns.regplot(x=clms1, y=clms2, data=df,ci=68,ax=axes[cnt1, cnt2])
        #sns.set_xlabel("X Label",fontsize=30)
        b.set_xlabel(clms2,fontsize=18*2)
        b.set_ylabel(clms1,fontsize=18*2)
        cnt2 +=1
    cnt1 += 1 
#plt.setp(axes, yticks=[])
plt.tight_layout()
plt.savefig('correlation_irt_avh.jpeg')

In [ ]:
# get a list of all columns in the dataframe without the Group column
df = opt_all[(opt_all['condition']=='unp')][['group','d','c','Ad','beta','fa']]
df.to_csv('tmp.csv')
column_list = [x for x in df.columns if x != 'group']
# create an empty dictionary
t_test_results = {}
# loop over column_list and execute code explained above
for column in column_list:
    group1 = df.where(df.group== 'High').dropna()[column]
    group2 = df.where(df.group== 'Low').dropna()[column]
    # add the output to the dictionary 
    t_test_results[column] = ttest_ind(group1,group2)
results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
results_df.columns = ['statistic','pvalue']
results_df

In [ ]:
opt_all_tmp = opt_all[(opt_all['condition']=='unp')][['Subject','group','d','c','Ad','beta','fa',
                                        'LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all',
                                         'QPE_severity','TAI_all','BDI_all',]]
# %store opt_all_tmp
# del opt_all_tmp

<font face="Arial" size=6 color=#0099ff>  8.Advanced machine learning approach. </font>

## To demostrate how easy to implement a machine learning model in python 

In [ ]:
opt_all_tmp.columns

In [ ]:
print opt_all_tmp.info()

In [ ]:
opt_all_tmp['HPSV_all'].fillna(X_train['HPSV_all'].mean(), inplace=True)
opt_all_tmp['RAHQ_all'].fillna(X_train['RAHQ_all'].mean(), inplace=True)
opt_all_tmp['QPE_severity'].fillna(X_train['QPE_severity'].mean(), inplace=True)
opt_all_tmp['TAI_all'].fillna(X_train['TAI_all'].mean(), inplace=True)
opt_all_tmp['BDI_all'].fillna(X_train['BDI_all'].mean(), inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
# selected_features = [u'd',u'c',u'Ad',u'beta',u'fa',u'LSHS_total',u'LSHS_AVH',\
#                      u'HPSV_all',u'RAHQ_all', u'QPE_severity',u'TAI_all',u'BDI_all']
selected_features = [u'd',u'c',u'Ad',u'beta',u'fa',u'LSHS_total',u'HPSV_all',u'RAHQ_all', u'QPE_severity',u'TAI_all',u'BDI_all']
X = opt_all_tmp[selected_features]
y = opt_all_tmp['group']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
print opt_all_tmp.info()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

from sklearn.model_selection import cross_val_score

cross_val_score(rfc, X_train, y_train, cv=5).mean()

In [ ]:
from xgboost import XGBClassifier

xgbc = XGBClassifier()

import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
cross_val_score(xgbc, X_train, y_train, cv=5).mean()

In [ ]:
rfc.fit(X_train,y_train)
rfc.feature_importances_

In [ ]:
range(10)

In [ ]:
len([range(10)])

In [ ]:
np.array([0.])
#np.concatenate((np.array([0.]), cdf, np.array([1.])))

In [ ]:
np.random.randn(len(range(10)))*(2**-1)

In [ ]:
xgbc.fit(X_train,y_train)
xgbc.feature_importances_